In [ ]:
import tensorflow as tf

# Sequence to Sequence 

2개의 RNN(recurrent neural networks)s로 이루어져 있다.

* Encoder maps a variable-length source sequence(input) to a fixed-length vector
* Decoder maps the vector representation back to a variable-length target sequence(output)
* Two RNNs are trained jointly to maximize the conditional probability of the target sequence given a source sequence

# With Attention 

* In the vanilla model, each input has to be encoded into a fixed-size state vector, as that is the only thing passed to the decoder.
* Attention mechanism that gives decoder direct access to the input.

# Bucketing 

* Avoid too much padding that leads to extraneous computation
* Group sequences of similar lengths into the same buckets
* Create a separate subgraph for each bucket

### v1.0 에 이런게 있다고 한다

#### tf.contrib.training.bucket_by_sequence_length(max_length,examples, batch_size, bucket_boundaries, capacity=2 * batch_size, dynamic_pad=True)


 Use bucket_by_sequence_length with the argument dynamic_pad=True to receive minibatches of similarly sized sequences for efficient training via dynamic_rnn.

# Sampled Softmax 

* Avoid the growing complexity of computing the normalization constant
* Approximate the negative term of the gradient, by importance sampling with a small number of samples
* At each step, update only the vectors associated with the correct word w and with the sampled words in V'
* Once training is over, use the full target vocabulary to compute the output probability of each target word

In [ ]:
if config.NUM_SAMPLES > 0 and config.NUM_SAMPLES < config.DEC_VOCAB:
    weight = tf.get_variable('proj_w', [config.HIDDEN_SIZE, config.DEC_VOCAB])
    bias = tf.get_variable('proj_b', [config.DEC_VOCAB])
    self.output_projection = (w, b)
    
def sampled_loss(inputs, labels):
    labels = tf.reshape(labels, [-1, 1])
    return tf.nn.sampled_softmax_loss(tf.transpose(weight), bias, inputs, labels,
                 config.NUM_SAMPLES, config.DEC_VOCAB)
self.softmax_loss_function = sampled_loss

### 실제 Predict할 때는 full softmax

* Generally an underestimate of the full softmax loss
* At inference time, compute the full softmax using:

In [ ]:
tf.nn.softmax(tf.matmul(inputs, tf.transpose(weight)) + bias)

# Seq2seq in tensorflow 

In [ ]:
outputs, states = basic_rnn_seq2seq(encoder_inputs, 
                                    decoder_inputs, 
                                    cell)

encoder_inputs: a list of tensors representing inputs to the encoder <br>
decoder_inputs: a list of tensors representing inputs to the decoder <br>
cell: single or multiple layer cells

outputs: a list of decoder_size tensors, each of dimension 1 x DECODE_VOCAB corresponding to the probability distribution at each time-step<br>
states: a list of decoder_size tensors, each corresponds to the internal state of the decoder at every time-step

### 근데 이 밑에 2개는 1.0에서 legacy가 되었네 ㅠ 

In [ ]:
outputs, states = embedding_rnn_seq2seq(encoder_inputs,
                                         decoder_inputs,
                                         cell,
                                         num_encoder_symbols,
                                         num_decoder_symbols,
                                         embedding_size,
                                         output_projection=None,
                                         feed_previous=False)


시퀀스 모델에 인풋과 아웃풋을 임베딩된 형태로 넘겨주기 위해선,input과 output의 심볼(토큰셋)의 수를 명시해줘야 한다. <br>
Output_projection: tuple of project weight and bias if use sampled softmax 

In [ ]:
outputs, states = embedding_attention_seq2seq(encoder_inputs,
                                             decoder_inputs,
                                             cell,
                                            num_encoder_symbols,
                                            num_decoder_symbols,
                                            num_heads=1,
                                            output_projection=None,
                                            feed_previous=False,
                                             initial_state_attention=False)


# Wrapper for seq2seq with buckets 

In [ ]:
outputs, losses = model_with_buckets(encoder_inputs,
                                     decoder_inputs,
                                     targets,
                                     weights,
                                     buckets,
                                     seq2seq,
                                     softmax_loss_function=None,
                                     per_example_loss=False)
